In [4]:
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, applications
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, ReduceLROnPlateau, Callback
from tensorflow.keras.mixed_precision import set_global_policy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

set_global_policy('mixed_float16')
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

data_dir = 'D:/Major Project/Dataset/Segmented_Augmented'
image_paths = []
labels = []
classes = sorted(os.listdir(data_dir))

for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    for img_name in os.listdir(class_dir):
        if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(class_dir, img_name)
            image_paths.append(img_path)
            labels.append(class_name)

df = pd.DataFrame({'image_path': image_paths, 'label': labels})
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

batch_size = 64
target_size = (224, 224)
num_classes = len(classes)

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='image_path',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3050 Laptop GPU, compute capability 8.6
Found 58400 validated image filenames belonging to 73 classes.
Found 7300 validated image filenames belonging to 73 classes.
Found 7300 validated image filenames belonging to 73 classes.


In [3]:
# Build initial model
base_model = applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
predictions = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)  

model = models.Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
print("Initial Model (All Base Layers Frozen):")
print(f"Total parameters: {model.count_params()}")
print(f"Trainable parameters: {sum([w.shape.num_elements() for w in model.trainable_weights])}")
print(f"Non-trainable parameters: {model.count_params() - sum([w.shape.num_elements() for w in model.trainable_weights])}")

# Define callbacks for initial training
csv_logger_initial = CSVLogger('D:/Major Project/mobilenet/training_history_initial.csv', separator=',', append=False)
early_stopping_initial = EarlyStopping(monitor='val_accuracy', patience=5, mode='max', restore_best_weights=True)
model_checkpoint_initial = ModelCheckpoint('D:/Major Project/mobilenet/best_initial_model.keras', monitor='val_accuracy', mode='max', save_best_only=True)

# Train initial model
steps_per_epoch = len(train_df) // batch_size
validation_steps = len(val_df) // batch_size

history_initial = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=50,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping_initial, model_checkpoint_initial, csv_logger_initial],
    workers=4,
    use_multiprocessing=False,
    verbose=1
)

Initial Model (All Base Layers Frozen):
Total parameters: 2431369
Trainable parameters: 173385
Non-trainable parameters: 2257984
Epoch 1/50
912/912 [==============================] - 355s 375ms/step - loss: 0.9446 - accuracy: 0.7170 - val_loss: 0.5592 - val_accuracy: 0.8206
Epoch 2/50
912/912 [==============================] - 329s 359ms/step - loss: 0.5121 - accuracy: 0.8263 - val_loss: 0.4346 - val_accuracy: 0.8513
Epoch 3/50
912/912 [==============================] - 313s 342ms/step - loss: 0.4408 - accuracy: 0.8468 - val_loss: 0.4326 - val_accuracy: 0.8494
Epoch 4/50
912/912 [==============================] - 314s 344ms/step - loss: 0.3936 - accuracy: 0.8618 - val_loss: 0.4152 - val_accuracy: 0.8565
Epoch 5/50
912/912 [==============================] - 313s 342ms/step - loss: 0.3701 - accuracy: 0.8692 - val_loss: 0.3585 - val_accuracy: 0.8757
Epoch 6/50
912/912 [==============================] - 316s 346ms/step - loss: 0.3467 - accuracy: 0.8769 - val_loss: 0.3381 - val_accuracy: 0.

In [6]:
steps_per_epoch = len(train_df) // batch_size
validation_steps = len(val_df) // batch_size
train_generator.reset()
val_generator.reset()
test_generator.reset()
print("Data generators reset to initial state")

def build_model(num_classes):
    base_model = applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    predictions = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
    model = models.Model(inputs=base_model.input, outputs=predictions)
    return model, base_model

num_classes = len(test_generator.class_indices) 
model, base_model = build_model(num_classes)

try:
    model.load_weights('D:/Major Project/mobilenet/best_initial_model.keras')
    print("Loaded initial training weights from 'best_initial_model.keras'")
except Exception as e:
    print(f"Error loading initial weights: {e}")
    print("Falling back to ImageNet weights")

for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Fine-Tuned Model (Last 20 Layers Unfrozen):")
print(f"Total parameters: {model.count_params()}")
print(f"Trainable parameters: {sum([w.shape.num_elements() for w in model.trainable_weights])}")
print(f"Non-trainable parameters: {model.count_params() - sum([w.shape.num_elements() for w in model.trainable_weights])}")

class SaveOnBestAccuracy(Callback):
    def __init__(self, filepath, monitor='val_accuracy', verbose=1):
        super(SaveOnBestAccuracy, self).__init__()
        self.filepath = filepath
        self.monitor = monitor
        self.verbose = verbose
        self.best_accuracy = -float('inf')

    def on_epoch_end(self, epoch, logs=None):
        current_accuracy = logs.get(self.monitor)
        if current_accuracy is None:
            print(f"Warning: {self.monitor} not available in logs")
            return
        if current_accuracy >= self.best_accuracy:
            self.best_accuracy = current_accuracy
            self.model.save(self.filepath, overwrite=True)
            if self.verbose:
                print(f"\nEpoch {epoch + 1}: {self.monitor} reached {current_accuracy:.4f}, saving model to {self.filepath}")

csv_logger_fine = CSVLogger('D:/Major Project/mobilenet/training_history_fine.csv', separator=',', append=False)
early_stopping_fine = EarlyStopping(monitor='val_accuracy', patience=5, mode='max', restore_best_weights=True)
model_checkpoint_fine = ModelCheckpoint(
    'D:/Major Project/mobilenet/best_fine_tuned_model.keras',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
save_best_accuracy = SaveOnBestAccuracy(
    filepath='D:/Major Project/mobilenet/best_fine_tuned_model_instant.keras',
    monitor='val_accuracy',
    verbose=1
)

history_fine = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=50,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping_fine, model_checkpoint_fine, csv_logger_fine, reduce_lr, save_best_accuracy],
    workers=4,
    use_multiprocessing=False,
    verbose=1
)

Data generators reset to initial state
Loaded initial training weights from 'best_initial_model.keras'
Fine-Tuned Model (Last 20 Layers Unfrozen):
Total parameters: 2431369
Trainable parameters: 1379465
Non-trainable parameters: 1051904
Epoch 1/50
912/912 [==============================] - ETA: 0s - loss: 0.6935 - accuracy: 0.7929
Epoch 1: val_accuracy improved from -inf to 0.88240, saving model to D:/Major Project/mobilenet\best_fine_tuned_model.keras

Epoch 1: val_accuracy reached 0.8824, saving model to D:/Major Project/mobilenet/best_fine_tuned_model_instant.keras
912/912 [==============================] - 366s 387ms/step - loss: 0.6935 - accuracy: 0.7929 - val_loss: 0.3691 - val_accuracy: 0.8824 - lr: 1.0000e-05
Epoch 2/50
912/912 [==============================] - ETA: 0s - loss: 0.3804 - accuracy: 0.8693
Epoch 2: val_accuracy improved from 0.88240 to 0.89953, saving model to D:/Major Project/mobilenet\best_fine_tuned_model.keras

Epoch 2: val_accuracy reached 0.8995, saving mode

In [18]:
model.save_weights('D:/Major Project/mobilenet/all_weights.keras')
model.save('D:/Major Project/mobilenet/final_model.keras')

In [10]:
# Evaluate model on test set
test_steps = len(test_df) // batch_size
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

114/114 [==============================] - 31s 270ms/step - loss: 0.1038 - accuracy: 0.9631
Test Loss: 0.1038, Test Accuracy: 0.9631


In [36]:
from sklearn.metrics import classification_report
import math

model = tf.keras.models.load_model('D:/Major Project/mobilenet/best_fine_tuned_model.keras')
print("Loaded fine-tuned model from 'best_fine_tuned_model.keras'")
test_generator.reset()
batch_size = 64
test_steps = math.ceil(len(test_df) / batch_size)
print(f"Test set size: {len(test_df)}, Batch size: {batch_size}, Test steps: {test_steps}")

test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps, verbose=1)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
y_pred = model.predict(test_generator, steps=test_steps, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)

y_true = test_generator.classes[:len(y_pred_classes)]  
class_labels = list(test_generator.class_indices.keys())

print(f"Length of y_true: {len(y_true)}")
print(f"Length of y_pred_classes: {len(y_pred_classes)}")
if len(y_true) != len(y_pred_classes):
    print("Warning: Mismatch in lengths. Truncating y_true to match y_pred_classes.")
    y_true = y_true[:len(y_pred_classes)]

cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(30, 30)) 
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels,
            cbar=False, linewidths=0.3, square=True)
plt.xticks(rotation=90, fontsize=7)
plt.yticks(fontsize=7)
plt.xlabel('Predicted Labels', fontsize=10)
plt.ylabel('True Labels', fontsize=10)
plt.title('Confusion Matrix', fontsize=14)
plt.tight_layout()

plt.savefig('D:/Major Project/mobilenet/confusion_matrix.png', dpi=300, bbox_inches='tight')  # Higher DPI for clarity
plt.close() 
print("Improved confusion matrix saved to 'D:/Major Project/mobilenet/confusion_matrix.png'")

Loaded fine-tuned model from 'best_fine_tuned_model.keras'
Test set size: 7300, Batch size: 64, Test steps: 115
115/115 [==============================] - 33s 277ms/step - loss: 0.1063 - accuracy: 0.9630
Test Loss: 0.1063, Test Accuracy: 0.9630
115/115 [==============================] - 32s 265ms/step
Length of y_true: 7300
Length of y_pred_classes: 7300
Improved confusion matrix saved to 'D:/Major Project/mobilenet/confusion_matrix.png'


In [37]:
model = tf.keras.models.load_model('D:/Major Project/mobilenet/best_fine_tuned_model.keras')
print("Loaded fine-tuned model from 'best_fine_tuned_model.keras'")
test_generator.reset()
batch_size = 64
test_steps = math.ceil(len(test_df) / batch_size)
print(f"Test set size: {len(test_df)}, Batch size: {batch_size}, Test steps: {test_steps}")
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps, verbose=1)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

y_pred = model.predict(test_generator, steps=test_steps, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes[:len(y_pred_classes)]
if len(y_true) != len(y_pred_classes):
    print("Warning: Mismatch in lengths. Truncating y_true to match y_pred_classes.")
    y_true = y_true[:len(y_pred_classes)]
cm = confusion_matrix(y_true, y_pred_classes)
max_classes = 20
if len(class_labels) > max_classes:
    class_labels = class_labels[:max_classes]
    cm = cm[:max_classes, :max_classes] 
    y_true = y_true[:len(y_pred_classes)]
    y_pred_classes = y_pred_classes[:len(y_pred_classes)]

plt.figure(figsize=(10, 10))  
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels,
            cbar=False, linewidths=0.3, square=True)

plt.xticks(rotation=90, fontsize=6)
plt.yticks(fontsize=6)
plt.xlabel('Predicted Labels', fontsize=8)
plt.ylabel('True Labels', fontsize=8)
plt.title('Confusion Matrix', fontsize=10)
plt.tight_layout()

plt.savefig('D:/Major Project/mobilenet/confusion_matrix_small.png', dpi=150, bbox_inches='tight')
plt.close()
print("Smaller confusion matrix saved to 'D:/Major Project/mobilenet/confusion_matrix_small.png'")

Loaded fine-tuned model from 'best_fine_tuned_model.keras'
Test set size: 7300, Batch size: 64, Test steps: 115
115/115 [==============================] - 32s 270ms/step - loss: 0.1063 - accuracy: 0.9630
Test Loss: 0.1063, Test Accuracy: 0.9630
115/115 [==============================] - 32s 271ms/step
Smaller confusion matrix saved to 'D:/Major Project/mobilenet/confusion_matrix_small.png'


In [48]:
import random
test_generator.reset()
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps, verbose=1)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

y_pred = model.predict(test_generator, steps=test_steps, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes[:len(y_pred_classes)]
if len(y_true) != len(y_pred_classes):
    print("Warning: Mismatch in lengths. Truncating y_true to match y_pred_classes.")
    y_true = y_true[:len(y_pred_classes)]
y_true = np.array(y_true)
y_pred_classes = np.array(y_pred_classes)

num_classes_to_show = 20
if len(class_labels) < num_classes_to_show:
    num_classes_to_show = len(class_labels)

random_classes = random.sample(class_labels, num_classes_to_show)
random_class_indices = [class_labels.index(cls) for cls in random_classes]
mask_true = np.isin(y_true, random_class_indices)

filtered_y_true_temp = y_true[mask_true]
filtered_y_pred_temp = y_pred_classes[mask_true]
mask_pred = np.isin(filtered_y_pred_temp, random_class_indices)

filtered_y_true = filtered_y_true_temp[mask_pred]
filtered_y_pred_classes = filtered_y_pred_temp[mask_pred]

label_mapping = {idx: i for i, idx in enumerate(random_class_indices)}
filtered_y_true_mapped = np.array([label_mapping[label] for label in filtered_y_true])
filtered_y_pred_mapped = np.array([label_mapping[label] for label in filtered_y_pred_classes])

cm = confusion_matrix(filtered_y_true_mapped, filtered_y_pred_mapped)

plt.figure(figsize=(10, 10)) 
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=random_classes, yticklabels=random_classes,
            cbar=False, linewidths=0.3, square=True)
plt.xticks(rotation=90, fontsize=6)
plt.yticks(fontsize=6)
plt.xlabel('Predicted Labels', fontsize=8)
plt.ylabel('True Labels', fontsize=8)
plt.title('Confusion Matrix', fontsize=12)
plt.tight_layout()

plt.savefig('D:/Major Project/mobilenet/confusion_matrix_random_subset.png', dpi=150, bbox_inches='tight')
plt.close()
print("Confusion matrix with random subset of classes saved to 'D:/Major Project/mobilenet/confusion_matrix_random_subset.png'")

115/115 [==============================] - 36s 307ms/step - loss: 0.1063 - accuracy: 0.9630
Test Loss: 0.1063, Test Accuracy: 0.9630
115/115 [==============================] - 34s 291ms/step
Confusion matrix with random subset of classes saved to 'D:/Major Project/mobilenet/confusion_matrix_random_subset.png'


In [44]:
class_labels = list(test_generator.class_indices.keys())
report = classification_report(y_true, y_pred_classes, target_names=class_labels, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv('D:/Major Project/mobilenet/classification_report.csv')
print("Classification report saved to 'D:/Major Project/mobilenet/classification_report.csv'")
print("\nClassification Report:")
print(report_df)

predictions_df = pd.DataFrame({
    'True_Label': [class_labels[i] for i in y_true],
    'Predicted_Label': [class_labels[i] for i in y_pred_classes],
    'Confidence': np.max(y_pred, axis=1)
})
predictions_df.to_csv('D:/Major Project/mobilenet/test_predictions.csv', index=False)
print("Test predictions saved to 'D:/Major Project/mobilenet/test_predictions.csv'")

Classification report saved to 'D:/Major Project/mobilenet/classification_report.csv'

Classification Report:
                              precision    recall  f1-score      support
Apple___Apple_scab             1.000000  0.990000  0.994975   100.000000
Apple___Black_rot              0.990000  0.990000  0.990000   100.000000
Apple___Cedar_apple_rust       0.990099  1.000000  0.995025   100.000000
Apple___healthy                0.990099  1.000000  0.995025   100.000000
Bittergourd__Downy_Mildew      0.914286  0.960000  0.936585   100.000000
...                                 ...       ...       ...          ...
Tomato___Tomato_mosaic_virus   0.989691  0.960000  0.974619   100.000000
Tomato___healthy               0.941748  0.970000  0.955665   100.000000
accuracy                       0.963014  0.963014  0.963014     0.963014
macro avg                      0.963144  0.963014  0.962739  7300.000000
weighted avg                   0.963144  0.963014  0.962739  7300.000000

[76 rows x 4 

In [46]:
try:
    history_initial_df = pd.read_csv('D:/Major Project/mobilenet/training_history_initial.csv')
    history_fine_df = pd.read_csv('D:/Major Project/mobilenet/training_history_fine_initial.csv')
    print("Loaded training history from 'training_history_initial.csv' and 'training_history_fine_initial.csv'")
except Exception as e:
    print(f"Error loading history CSVs: {e}")
    print("Please ensure 'training_history_initial.csv' and 'training_history_fine_initial.csv' exist in 'D:/Major Project/mobilenet/'")
    exit(1)

required_columns = ['accuracy', 'val_accuracy', 'loss', 'val_loss']
if not all(col in history_initial_df.columns for col in required_columns) or \
   not all(col in history_fine_df.columns for col in required_columns):
    print("Error: CSV files must contain columns: 'accuracy', 'val_accuracy', 'loss', 'val_loss'")
    exit(1)

history_initial = {
    'accuracy': history_initial_df['accuracy'].tolist(),
    'val_accuracy': history_initial_df['val_accuracy'].tolist(),
    'loss': history_initial_df['loss'].tolist(),
    'val_loss': history_initial_df['val_loss'].tolist()
}
history_fine = {
    'accuracy': history_fine_df['accuracy'].tolist(),
    'val_accuracy': history_fine_df['val_accuracy'].tolist(),
    'loss': history_fine_df['loss'].tolist(),
    'val_loss': history_fine_df['val_loss'].tolist()
}
print(f"Initial training epochs: {len(history_initial['accuracy'])}")
print(f"Fine-tuning epochs: {len(history_fine['accuracy'])}")
print(f"Total epochs: {len(history_initial['accuracy']) + len(history_fine['accuracy'])}")

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_initial['accuracy'] + history_fine['accuracy'], label='Train Accuracy')
plt.plot(history_initial['val_accuracy'] + history_fine['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_initial['loss'] + history_fine['loss'], label='Train Loss')
plt.plot(history_initial['val_loss'] + history_fine['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('D:/Major Project/mobilenet/training_history.png')
plt.close()
print("Training history plot saved to 'D:/Major Project/mobilenet/training_history.png'")

Loaded training history from 'training_history_initial.csv' and 'training_history_fine_initial.csv'
Initial training epochs: 23
Fine-tuning epochs: 50
Total epochs: 73
Training history plot saved to 'D:/Major Project/mobilenet/training_history.png'
